## Nei RDBMS classici...

- un database aggrega delle tabelle
- una tabella aggrega delle righe
- una riga aggrega dei valori per attributi (fissi)
- si eseguono operazioni di join tra tabelle
- la struttura delle righe in una tabella è prefissata (schema)

## In mongoDB...

- un database aggrega delle <span style="text-decoration: line-through;">tabelle</span> collezioni
- una <span style="text-decoration: line-through;">tabella</span> collezione aggrega <span style="text-decoration: line-through;">delle righe</span> dei documenti
- <span style="text-decoration: line-through;">una riga</span> un documento aggrega dei <span style="text-decoration: line-through;">valori per attributi (fissi)</span> campi (variabili) indicati tramite coppie (nome, valore)
- <span style="text-decoration: line-through;">si eseguono</span> non si possono eseguire (direttamente) operazioni di join tra tabelle
- la struttura delle righe in una tabella **non** è prefissata <span style="text-decoration: line-through;">(schema)</span>

## In MongoDB

- Invece di memorizzare l'informazione nelle righe di una tabella, MongoDB memorizza dati strutturati utilizzando un formato simile a JSON
- JSON (JavaScript Object Notation, http://www.json.org) è un formato lightweight per lo scambio di dati, facile da leggere (per gli umani) e da analizzare (dalle macchine)

## Un esempio

- La tabella

|Nome    |Cognome |Targa|
|--------|------- |-----|
|Paolino |Paperino|  313|
|Topolino|        |  113|

- diventa

In [1]:
{'nome': 'Paolino',
 'cognome': 'Paperino',
 'targa': 313},
{'nome': 'Topolino',
 'targa': 113}

{'nome': 'Topolino', 'targa': 113}

## JSON e BSON

Il formato in cui effettivamente MongoDB memorizza l'informazione è BSON (Binary JSON, http://bsonspec.org/), che permette di

- serializzare documenti JSON in modo efficiente
- attraversare facilmente tali documenti
- conservare la lightweightness

## Documenti incorporati

È possibile inserire un documento all'interno di un campo BSON

In [2]:
{'nome': 'Paolino',
 'cognome': 'Paperino',
 'auto': {'modello': 'American Bantam', 'targa': 313}
}

{'auto': {'modello': 'American Bantam', 'targa': 313},
 'cognome': 'Paperino',
 'nome': 'Paolino'}

## Array

In BSON gli array sono oggetti di prima classe

In [3]:
{'nome': 'Paolino',
 'cognome': 'Paperino',
 'auto': {'modello': 'American Bantam', 'targa': 313},
 'familiari': ['qui', 'quo', 'qua']
}

{'auto': {'modello': 'American Bantam', 'targa': 313},
 'cognome': 'Paperino',
 'familiari': ['qui', 'quo', 'qua'],
 'nome': 'Paolino'}

## Prime operazioni con MongoDB

- Per poter interagire con MongoDB è necessario generare una _connessione_

In [9]:
from pymongo import MongoClient
mongo = MongoClient(host='127.0.0.1', port=27017)

- è durante questa fase che si effettua l'eventuale autenticazione

## Prime operazioni con MongoDB

- si ottiene un riferimento a un DB a partire dalla connessione usando una _dot notation_ 

In [10]:
db = mongo.fumetti

- in alternativa è possibile usare una notazione analoga a quella dei dizionari

In [11]:
mongo['fumetti']

Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'fumetti')

- in entrambi i casi, qualora non esista il DB viene creato

## Prime operazioni con MongoDB

- Analogamente, a partire dal riferimento al DB si ottiene tramite una delle due sintassi viste il riferimento a una collezione

In [17]:
db.personaggi

Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'fumetti'), 'personaggi')

In [18]:
db['personaggi']

Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'fumetti'), 'personaggi')

## Prime operazioni con MongoDB

- Inserimento di un documento (con contestuale creazione della collezione)

In [19]:
res = db.personaggi.insert_one({'nome': 'Paolino',
                                'cognome': 'Paperino',
                                'genere': 'M',
                                'auto': {'modello': 'American Bantam', 'targa': 313},
                                'anno_nascita': 1920,
                                'prima_apparizione': 1934})

In [20]:
res.inserted_id

ObjectId('5cfe7911fc4b171de3b919a5')

## Prime operazioni con MongoDB

- La dimensione massima di un documento è di 64 MByte
- L'inserimento è possibile anche attraverso _upsert_ (vedi oltre)

## Prime operazioni con MongoDB

- Lettura di una collezione

In [21]:
list(db.personaggi.find())

[{'_id': ObjectId('5cfe7911fc4b171de3b919a5'),
  'nome': 'Paolino',
  'cognome': 'Paperino',
  'genere': 'M',
  'auto': {'modello': 'American Bantam', 'targa': 313},
  'anno_nascita': 1920,
  'prima_apparizione': 1934}]

## Prime operazioni con MongoDB

- lettura di un elemento in una collezione (**non** necessariamente il primo)

In [22]:
db.personaggi.find_one()

{'_id': ObjectId('5cfe7911fc4b171de3b919a5'),
 'nome': 'Paolino',
 'cognome': 'Paperino',
 'genere': 'M',
 'auto': {'modello': 'American Bantam', 'targa': 313},
 'anno_nascita': 1920,
 'prima_apparizione': 1934}

## Identificatori univoci

- Il campo `_id` contiene un valore generato automaticamente in termini di `ObjectId`

In [23]:
from bson.objectid import ObjectId

[ObjectId() for _ in range(3)]

[ObjectId('5cfe791dfc4b171de3b919a6'),
 ObjectId('5cfe791dfc4b171de3b919a7'),
 ObjectId('5cfe791dfc4b171de3b919a8')]

- MongoDB indicizza automaticamente i documenti inseriti in una collezione relativamente al campo `_id`

## Popoliamo una collezione

In [24]:
paperi = [{'nome': 'nonna papera', 'genere': 'F',
  'auto': {'modello': 'Detroit Electric'},
  'anno_nascita': 1833, 'prima_apparizione': 1943,
  'passatempi': ['cucina', 'agricoltura']},
 {'nome': 'Paperon', 'cognome': 'De Paperoni', 'genere': 'M',
  'anno_nascita': 1867,   'prima_apparizione': 1947,
  'passatempi': ['alta finanza', 'risparmio', 'nuoto in depositi aurei']},
 {'nome': 'Archimede', 'cognome': 'Pitagorico', 'genere': 'M',
  'prima_apparizione': 1952,
  'passatempi': ['invenzioni', 'studio']},
 {'nome': 'Pico', 'cognome': 'De Paperis', 'genere': 'M',
  'prima_apparizione': 1961,
  'passatempi': ['studio']}]

db.personaggi.insert_many(paperi)

- Il metodo `insert_many` restituisce un oggetto della classe `InsertManyResult` la cui variabile di istanza `inserted_ids` contiene gli `ObjectId` di tutti i documenti inseriti

## Selettori semplici

- in base a esistenza di campi

In [25]:
list(db.personaggi.find({'auto': {'$exists': True}}))

[{'_id': ObjectId('5cfe7911fc4b171de3b919a5'),
  'nome': 'Paolino',
  'cognome': 'Paperino',
  'genere': 'M',
  'auto': {'modello': 'American Bantam', 'targa': 313},
  'anno_nascita': 1920,
  'prima_apparizione': 1934},
 {'_id': ObjectId('5cfe7923fc4b171de3b919a9'),
  'nome': 'nonna papera',
  'genere': 'F',
  'auto': {'modello': 'Detroit Electric'},
  'anno_nascita': 1833,
  'prima_apparizione': 1943,
  'passatempi': ['cucina', 'agricoltura']}]

## Selettori semplici

- in base a valori specifici

In [26]:
list(db.personaggi.find({'genere': 'F'}))

[{'_id': ObjectId('5cfe7923fc4b171de3b919a9'),
  'nome': 'nonna papera',
  'genere': 'F',
  'auto': {'modello': 'Detroit Electric'},
  'anno_nascita': 1833,
  'prima_apparizione': 1943,
  'passatempi': ['cucina', 'agricoltura']}]

In [27]:
list(db.personaggi.find({'prima_apparizione': 1961}))

[{'_id': ObjectId('5cfe7923fc4b171de3b919ac'),
  'nome': 'Pico',
  'cognome': 'De Paperis',
  'genere': 'M',
  'prima_apparizione': 1961,
  'passatempi': ['studio']}]

## Selettori semplici

- in base a una relazione

In [28]:
list(db.personaggi.find({'prima_apparizione': {'$gt': 1950}}))

[{'_id': ObjectId('5cfe7923fc4b171de3b919ab'),
  'nome': 'Archimede',
  'cognome': 'Pitagorico',
  'genere': 'M',
  'prima_apparizione': 1952,
  'passatempi': ['invenzioni', 'studio']},
 {'_id': ObjectId('5cfe7923fc4b171de3b919ac'),
  'nome': 'Pico',
  'cognome': 'De Paperis',
  'genere': 'M',
  'prima_apparizione': 1961,
  'passatempi': ['studio']}]

## Selettori semplici

- in base alla presenza di valori in un insieme

In [29]:
list(db.personaggi.find({'prima_apparizione': {'$in': [1947, 1961]}}))

[{'_id': ObjectId('5cfe7923fc4b171de3b919aa'),
  'nome': 'Paperon',
  'cognome': 'De Paperoni',
  'genere': 'M',
  'anno_nascita': 1867,
  'prima_apparizione': 1947,
  'passatempi': ['alta finanza', 'risparmio', 'nuoto in depositi aurei']},
 {'_id': ObjectId('5cfe7923fc4b171de3b919ac'),
  'nome': 'Pico',
  'cognome': 'De Paperis',
  'genere': 'M',
  'prima_apparizione': 1961,
  'passatempi': ['studio']}]

## Selettori complessi

- in base a congiunzioni logiche

In [30]:
list(db.personaggi.find({'prima_apparizione': {'$lt': 1950},
                         'genere': 'M'}))

[{'_id': ObjectId('5cfe7911fc4b171de3b919a5'),
  'nome': 'Paolino',
  'cognome': 'Paperino',
  'genere': 'M',
  'auto': {'modello': 'American Bantam', 'targa': 313},
  'anno_nascita': 1920,
  'prima_apparizione': 1934},
 {'_id': ObjectId('5cfe7923fc4b171de3b919aa'),
  'nome': 'Paperon',
  'cognome': 'De Paperoni',
  'genere': 'M',
  'anno_nascita': 1867,
  'prima_apparizione': 1947,
  'passatempi': ['alta finanza', 'risparmio', 'nuoto in depositi aurei']}]

## Selettori complessi

- in base a disgiunzioni logiche

In [31]:
list(db.personaggi.find({'$or': [{'anno_nascita': {'$gt': 1900}},
                                {'genere': 'F'}]}))

[{'_id': ObjectId('5cfe7911fc4b171de3b919a5'),
  'nome': 'Paolino',
  'cognome': 'Paperino',
  'genere': 'M',
  'auto': {'modello': 'American Bantam', 'targa': 313},
  'anno_nascita': 1920,
  'prima_apparizione': 1934},
 {'_id': ObjectId('5cfe7923fc4b171de3b919a9'),
  'nome': 'nonna papera',
  'genere': 'F',
  'auto': {'modello': 'Detroit Electric'},
  'anno_nascita': 1833,
  'prima_apparizione': 1943,
  'passatempi': ['cucina', 'agricoltura']}]

## Selettori complessi

- in base a occorrenze in un di array

In [32]:
list(db.personaggi.find({'passatempi': 'studio'}))

[{'_id': ObjectId('5cfe7923fc4b171de3b919ab'),
  'nome': 'Archimede',
  'cognome': 'Pitagorico',
  'genere': 'M',
  'prima_apparizione': 1952,
  'passatempi': ['invenzioni', 'studio']},
 {'_id': ObjectId('5cfe7923fc4b171de3b919ac'),
  'nome': 'Pico',
  'cognome': 'De Paperis',
  'genere': 'M',
  'prima_apparizione': 1961,
  'passatempi': ['studio']}]

## Selettori complessi

- in base alla presenza di un intero array

In [33]:
list(db.personaggi.find({'passatempi': ['studio']}))

[{'_id': ObjectId('5cfe7923fc4b171de3b919ac'),
  'nome': 'Pico',
  'cognome': 'De Paperis',
  'genere': 'M',
  'prima_apparizione': 1961,
  'passatempi': ['studio']}]

## Selettori complessi

- in base a occorrenze in posizioni specifiche di un array

In [34]:
list(db.personaggi.find({'passatempi.1': 'studio'}))

[{'_id': ObjectId('5cfe7923fc4b171de3b919ab'),
  'nome': 'Archimede',
  'cognome': 'Pitagorico',
  'genere': 'M',
  'prima_apparizione': 1952,
  'passatempi': ['invenzioni', 'studio']}]

## Selettori complessi

- in base al contenuto di documenti incorporati

In [35]:
list(db.personaggi.find({'auto.modello': 'American Bantam'}))

[{'_id': ObjectId('5cfe7911fc4b171de3b919a5'),
  'nome': 'Paolino',
  'cognome': 'Paperino',
  'genere': 'M',
  'auto': {'modello': 'American Bantam', 'targa': 313},
  'anno_nascita': 1920,
  'prima_apparizione': 1934}]

## Selettori complessi

- in base a espressioni regolari

In [36]:
import re
regx = re.compile("^.*aper", re.IGNORECASE)

list(db.personaggi.find({'nome': regx}))

[{'_id': ObjectId('5cfe7923fc4b171de3b919a9'),
  'nome': 'nonna papera',
  'genere': 'F',
  'auto': {'modello': 'Detroit Electric'},
  'anno_nascita': 1833,
  'prima_apparizione': 1943,
  'passatempi': ['cucina', 'agricoltura']},
 {'_id': ObjectId('5cfe7923fc4b171de3b919aa'),
  'nome': 'Paperon',
  'cognome': 'De Paperoni',
  'genere': 'M',
  'anno_nascita': 1867,
  'prima_apparizione': 1947,
  'passatempi': ['alta finanza', 'risparmio', 'nuoto in depositi aurei']}]

## Selettori complessi

In base a relazioni di carattere geospaziale

- `$near` permette di individuare punti che si trovano a una distanza radiale compresa tra un minimo e un massimo rispetto a un dato punto

- `$within` permette di individuare punti che giacciono all'interno di una regione (per esempio indicata attraverso un poligono)

## Selezioni ed efficienza

L'uso di espressioni regolari e di altri strumenti come `$lt` e `$gt` può essere fonte di inefficienza durante l'esecuzione delle query

## Proiezioni

È possibile visualizzare campi specifici dei documenti selezionati specificando un secondo argomento per `find` e `find_one`

- contenente un elenco dei campi da visualizzare

In [37]:
list(db.personaggi.find({'passatempi.1': 'studio'},
                        {'nome': 1, 'cognome': 1}))

[{'_id': ObjectId('5cfe7923fc4b171de3b919ab'),
  'nome': 'Archimede',
  'cognome': 'Pitagorico'}]

## Proiezioni

È possibile visualizzare campi specifici dei documenti selezionati specificando un secondo argomento per `find` e `find_one`

- o di quelli da escludere

In [38]:
list(db.personaggi.find({'passatempi.1': 'studio'},
                        {'cognome': 0,
                         'genere': 0,
                         'anno_nascita': 0,
                         'prima_apparizione': 0,
                         'passatempi': 0}))

[{'_id': ObjectId('5cfe7923fc4b171de3b919ab'), 'nome': 'Archimede'}]

## Proiezioni

Non è possibile utilizzare i due approcci assieme, con l'unica eccezione dell'esclusione di `_id`, che altrimenti viene sempre visualizzato

In [39]:
list(db.personaggi.find({"passatempi.1": 'studio'},
                        {'nome': 1, 'cognome': 1, '_id': 0}))

[{'nome': 'Archimede', 'cognome': 'Pitagorico'}]

## Indici

- È possibile associare un indice a una collezione

In [40]:
import pymongo
db.personaggi.create_index([('nome', pymongo.ASCENDING),
                            ('prima_apparizione', pymongo.DESCENDING)])

'nome_1_prima_apparizione_-1'

- l'indice è associato a un campo o a un gruppo di campi, ordinati in modo non decrescente (1) o non crescente (-1)

## Indici

- un indice ricopre una query se di esso fanno parte
  - tutti i campi filtrati dalla query, e
  - tutti i campi restituiti dalla query
- le query ricoperte vengono eseguite leggendo l'indice

In [41]:
regx = re.compile("^P", re.IGNORECASE)

list(db.personaggi.find({'nome': regx,
                    'prima_apparizione': {'$lt': 1950}},
                   {'_id': 0, 'nome': 1, 'prima_apparizione': 1}))

[{'nome': 'Paolino', 'prima_apparizione': 1934},
 {'nome': 'Paperon', 'prima_apparizione': 1947}]

## Cursori

`find` restituisce un cursore ai suoi risultati, il quale può essere acceduto

- nelle modalità con cui di solito si utilizza un _iterable_

## Cursori

`find` restituisce un cursore ai suoi risultati, il quale può essere acceduto

- usando il metodo `next`

In [42]:
cursor = db.personaggi.find()
personaggio = cursor.next()

personaggio['auto']

{'modello': 'American Bantam', 'targa': 313}

In [43]:
personaggio['auto']['targa']

313

## Cursori

`find` restituisce un cursore ai suoi risultati, il quale può essere acceduto

- accedendo alla variabile di istanza `alive` per verificare se esistano risultati che non sono ancora stati considerati

In [44]:
cursor.alive

True

## Cursori

`find` restituisce un cursore ai suoi risultati, il quale può essere acceduto

- usando una notazione posizionale (solo se non si è già acceduto ai risultati)

In [45]:
cursor = db.personaggi.find()
cursor[2] # se è già stato usato non funziona

{'_id': ObjectId('5cfe7923fc4b171de3b919aa'),
 'nome': 'Paperon',
 'cognome': 'De Paperoni',
 'genere': 'M',
 'anno_nascita': 1867,
 'prima_apparizione': 1947,
 'passatempi': ['alta finanza', 'risparmio', 'nuoto in depositi aurei']}

## Metodi sui cursori

- `limit` limita il numero di risultati

In [46]:
# equivale a find()[:3]

list(db.personaggi.find().limit(3))

[{'_id': ObjectId('5cfe7911fc4b171de3b919a5'),
  'nome': 'Paolino',
  'cognome': 'Paperino',
  'genere': 'M',
  'auto': {'modello': 'American Bantam', 'targa': 313},
  'anno_nascita': 1920,
  'prima_apparizione': 1934},
 {'_id': ObjectId('5cfe7923fc4b171de3b919a9'),
  'nome': 'nonna papera',
  'genere': 'F',
  'auto': {'modello': 'Detroit Electric'},
  'anno_nascita': 1833,
  'prima_apparizione': 1943,
  'passatempi': ['cucina', 'agricoltura']},
 {'_id': ObjectId('5cfe7923fc4b171de3b919aa'),
  'nome': 'Paperon',
  'cognome': 'De Paperoni',
  'genere': 'M',
  'anno_nascita': 1867,
  'prima_apparizione': 1947,
  'passatempi': ['alta finanza', 'risparmio', 'nuoto in depositi aurei']}]

## Metodi sui cursori

- `skip` sposta il punto in cui il cursore inizia a restituire risultati

In [47]:
# equivale a find()[2:]

list(db.personaggi.find().skip(2))

[{'_id': ObjectId('5cfe7923fc4b171de3b919aa'),
  'nome': 'Paperon',
  'cognome': 'De Paperoni',
  'genere': 'M',
  'anno_nascita': 1867,
  'prima_apparizione': 1947,
  'passatempi': ['alta finanza', 'risparmio', 'nuoto in depositi aurei']},
 {'_id': ObjectId('5cfe7923fc4b171de3b919ab'),
  'nome': 'Archimede',
  'cognome': 'Pitagorico',
  'genere': 'M',
  'prima_apparizione': 1952,
  'passatempi': ['invenzioni', 'studio']},
 {'_id': ObjectId('5cfe7923fc4b171de3b919ac'),
  'nome': 'Pico',
  'cognome': 'De Paperis',
  'genere': 'M',
  'prima_apparizione': 1961,
  'passatempi': ['studio']}]

## Metodi sui cursori

- un mix di `skip` e `limit` equivale a un _list slicing_ sul cursore

- `count` conta il numero di documenti in un risultato

In [48]:
db.personaggi.find().count()

/home/malchiodi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


5

## Metodi sui cursori

- `sort` ordina i risultati

In [49]:
list(db.personaggi.find().sort([('cognome', pymongo.ASCENDING),
                                ('nome', pymongo.ASCENDING)]))

[{'_id': ObjectId('5cfe7923fc4b171de3b919a9'),
  'nome': 'nonna papera',
  'genere': 'F',
  'auto': {'modello': 'Detroit Electric'},
  'anno_nascita': 1833,
  'prima_apparizione': 1943,
  'passatempi': ['cucina', 'agricoltura']},
 {'_id': ObjectId('5cfe7923fc4b171de3b919ac'),
  'nome': 'Pico',
  'cognome': 'De Paperis',
  'genere': 'M',
  'prima_apparizione': 1961,
  'passatempi': ['studio']},
 {'_id': ObjectId('5cfe7923fc4b171de3b919aa'),
  'nome': 'Paperon',
  'cognome': 'De Paperoni',
  'genere': 'M',
  'anno_nascita': 1867,
  'prima_apparizione': 1947,
  'passatempi': ['alta finanza', 'risparmio', 'nuoto in depositi aurei']},
 {'_id': ObjectId('5cfe7911fc4b171de3b919a5'),
  'nome': 'Paolino',
  'cognome': 'Paperino',
  'genere': 'M',
  'auto': {'modello': 'American Bantam', 'targa': 313},
  'anno_nascita': 1920,
  'prima_apparizione': 1934},
 {'_id': ObjectId('5cfe7923fc4b171de3b919ab'),
  'nome': 'Archimede',
  'cognome': 'Pitagorico',
  'genere': 'M',
  'prima_apparizione': 1952,

## Aggiornamento

- il metodo `update_one` permette di modificare un documento

In [50]:
res = db.personaggi.update_one({'nome': 'Archimede'},
                         {'$set': {'prima_apparizione': 1950}})

-  (**non** si ha il controllo su quale dei documenti individuati verrà effettuata la modifica)
- il metodo `update_many` modifica un insieme di documenti
- nelle prime versioni di MongoDB sono possibili query del tipo

  ```
  db.personaggi.update_one({'nome': 'Archimede},
                           {'prima_apparizione': 1950}) ```

  che **rimpiazzano** l'intero documento!

## Aggiornamento

Le query di aggiornamento restituiscono un oggetto della classe `UpdateResult` con le seguenti variabili di istanza

- `raw_result`, che contiene un dizionario che riassume i risultati dell'aggiornamento

In [51]:
res.raw_result

{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}

## Aggiornamento

Le query di aggiornamento restituiscono un oggetto della classe `UpdateResult` con le seguenti variabili di istanza

- `matched_count`, che indica il numero di documenti che hanno soddisfatto il criterio di ricerca

In [52]:
res.matched_count

1

## Aggiornamento

Le query di aggiornamento restituiscono un oggetto della classe `UpdateResult` con le seguenti variabili di istanza

- `modified_count`, che indica quanti documenti sono stati effettivamente modificati

In [53]:
res.modified_count

1

## Aggiornamento

- `$inc` incrementa le quantità numeriche

- `$push` aggiunge elementi a un array

In [54]:
_ = db.personaggi.update_one({'nome': 'Archimede'},
                             {'$push': {'passatempi': 'lampadine'}})

In [55]:
db.personaggi.find_one({'nome': 'Archimede'})

{'_id': ObjectId('5cfe7923fc4b171de3b919ab'),
 'nome': 'Archimede',
 'cognome': 'Pitagorico',
 'genere': 'M',
 'prima_apparizione': 1950,
 'passatempi': ['invenzioni', 'studio', 'lampadine']}

## Aggiornamento

- `$pull` rimuove tutte le occorrenze di un elemento da un array

In [56]:
db.personaggi.update_one({'nome': 'Archimede'},
                         {'$pull': {'passatempi': 'lampadine'}})

In [57]:
db.personaggi.find_one({'nome': 'Archimede'})

{'_id': ObjectId('5cfe7923fc4b171de3b919ab'),
 'nome': 'Archimede',
 'cognome': 'Pitagorico',
 'genere': 'M',
 'prima_apparizione': 1950,
 'passatempi': ['invenzioni', 'studio']}

## Peculiarità di `update`


- impostando a `True` il terzo argomento di `update_one` o `update_many`, l'operazione di aggiornamento crea un documento qualora quello da modificare non esista (_upsert_)

In [58]:
db.personaggi.update_one({'nome': 'Archimede'},
                         {'$pull': {'passatempi': 'lampadine'}},
                         upsert=True)

## Cancellazione

- Il metodo `delete_one` cancella un documento

In [59]:
res = db.personaggi.delete_one({'nome': 'Poco'})

- sull'oggetto restituito è possibile leggere le variabili di istanza `raw_result` e `deleted_count`

In [60]:
res.raw_result

{'n': 0, 'ok': 1.0}

In [61]:
res.deleted_count

0

## Cancellazione

- analogamente, `delete_many` cancella un insieme di documenti

In [62]:
res = db.personaggi.delete_many({'passatempi': 'fumetti'})
res.raw_result

{'n': 0, 'ok': 1.0}

- in teoria è ancora supportato il metodo `remove`, che però è deprecato

## Cancellazione di collezioni e DB

- La cancellazione di una collezione si effettua invocando il metodo `drop_collection` sull'oggetto DB o il metodo `drop` sull'oggetto della collezione stessa

In [ ]:
# db.fumetti.drop_collection('personaggi')

# db.fumetti.personaggi.drop()

- La cancellazione di un database si effettua invocando il metodo `drop_database` sull'oggetto ottenuto in fase di connessione

In [135]:
#mongo.drop_database('fumetti')

## Map-reduce

```
resource    date
index       Jan 20 2010 4:30
index       Jan 20 2010 5:30
about       Jan 20 2010 6:00
index       Jan 20 2010 7:00
about       Jan 21 2010 8:00
about       Jan 21 2010 8:30
index       Jan 21 2010 8:30
about       Jan 21 2010 9:00
index       Jan 21 2010 9:30
index       Jan 22 2010 5:00
```

Partendo da questi dati...

## Map-reduce

...vogliamo ottenere

```
resource    year    month   day     count
index       2010    1       20      3
about       2010    1       20      1
about       2010    1       21      3
index       2010    1       21      2
index       2010    1       22      1
```

Usiamo un algoritmo Map-reduce in cui

- map emette 1 per ogni chiave (risorsa, data)
- reduce somma i valori

## Map-reduce

popoliamo il database

In [136]:
import datetime

db.hits.insert_one({'resource': 'index', 'date': datetime.datetime(2010, 1, 20, 4, 30, 0, 0)})
db.hits.insert_one({'resource': 'index', 'date': datetime.datetime(2010, 1, 20, 5, 30, 0, 0)})
db.hits.insert_one({'resource': 'about', 'date': datetime.datetime(2010, 1, 20, 6, 0, 0, 0)})
db.hits.insert_one({'resource': 'index', 'date': datetime.datetime(2010, 1, 20, 7, 0, 0, 0)})
db.hits.insert_one({'resource': 'about', 'date': datetime.datetime(2010, 1, 21, 8, 0, 0, 0)})
db.hits.insert_one({'resource': 'index', 'date': datetime.datetime(2010, 1, 21, 8, 30, 0, 0)})
db.hits.insert_one({'resource': 'about', 'date': datetime.datetime(2010, 1, 21, 9, 0, 0, 0)})
db.hits.insert_one({'resource': 'index', 'date': datetime.datetime(2010, 1, 21, 9, 30, 0, 0)})
_ = db.hits.insert_one({'resource': 'index', 'date': datetime.datetime(2010, 1, 22, 5, 0, 0, 0)})

## Map-reduce

creiamo la funzione map (in javascript!)

In [93]:
from bson.code import Code

map = Code('''function() {
  var key = {
    resource: this.resource,
    year: this.date.getFullYear(),
    month: this.date.getMonth(),
    day: this.date.getDate()
  };
  emit(key, {count: 1});
}''')

## Map-reduce

creiamo la funzione reduce

In [94]:
reduce = Code('''function(key, values) {
  var sum = 0;
  values.forEach(function(value) {
     sum += value['count'];
  });
  return {count: sum};
}''')

## Map-reduce

_et voilà_

In [95]:
result = db.hits.map_reduce(map, reduce, 'hit_stats')

In [96]:
result

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'fumetti'), u'hit_stats')

In [97]:
for doc in result.find():
    print doc

{u'_id': {u'month': 0.0, u'resource': u'about', u'day': 20.0, u'year': 2010.0}, u'value': {u'count': 1.0}}
{u'_id': {u'month': 0.0, u'resource': u'about', u'day': 21.0, u'year': 2010.0}, u'value': {u'count': 2.0}}
{u'_id': {u'month': 0.0, u'resource': u'index', u'day': 20.0, u'year': 2010.0}, u'value': {u'count': 3.0}}
{u'_id': {u'month': 0.0, u'resource': u'index', u'day': 21.0, u'year': 2010.0}, u'value': {u'count': 2.0}}
{u'_id': {u'month': 0.0, u'resource': u'index', u'day': 22.0, u'year': 2010.0}, u'value': {u'count': 1.0}}


## Altre forme di aggregazione

MongoDB prevede forme più deboli di aggregazione che non richiedono l'utilizzo di algoritmi map-reduce, utilizzando il metodo `aggregate` in congiunzione con operatori come `$match`, `$project`, `$group`, `$sort`, ecc.

## Join

- i database orientati ai documenti sono forse quelli più vicini all'approccio relazionale, ma...
- le operazioni di join non vanno a braccetto con la scalabilità orizzontale

## Join

Nessuno ci impedisce di fare le join "a mano"...

In [141]:
id_zio = db.personaggi.find_one({'cognome': 'Paperino'})['_id']

db.personaggi.insert_one({'nome': 'Qui', 'capofamiglia': id_zio})
db.personaggi.insert_one({'nome': 'Quo', 'capofamiglia': id_zio})
db.personaggi.insert_one({'nome': 'Qua', 'capofamiglia': id_zio})

...al prezzo di una query in più (per recuperare il capofamiglia).

## Relazioni molti a molti

- Si possono utilizzare gli array

In [147]:
id_qui = db.personaggi.find_one({'nome': 'Qui'})['_id']
id_quo = db.personaggi.find_one({'nome': 'Quo'})['_id']
id_qua = db.personaggi.find_one({'nome': 'Qua'})['_id']

db.personaggi.update_one({'nome': 'Paolino', 'cognome': 'Paperino'},
                         {'$set': {'familiari': [id_qui, id_quo, id_qua]}})

db.personaggi.find_one({'familiari': id_quo})

{u'_id': ObjectId('5a06255f8eac92b1830c46f2'),
 u'anno_nascita': 1920,
 u'auto': {u'modello': u'American Bantam', u'targa': 313},
 u'cognome': u'Paperino',
 u'familiari': [ObjectId('5a0737428eac92b1830c4704'),
  ObjectId('5a0737428eac92b1830c4705'),
  ObjectId('5a0737428eac92b1830c4706')],
 u'genere': u'M',
 u'nome': u'Paolino',
 u'prima_apparizione': 1934}

## Relazioni molti a molti

- o i documenti incorporati

In [169]:
db.personaggi.update_one({'nome': 'Paolino', 'cognome': 'Paperino'},
                         {'$set': {'familiari': [{'nome': 'Qui'},
                                                 {'nome': 'Quo'},
                                                 {'nome': 'Qua'}]}})

db.personaggi.find_one({'familiari.nome': 'Quo'})

{u'_id': ObjectId('5a0739448eac92b1830c4707'),
 u'anno_nascita': 1920,
 u'auto': {u'modello': u'American Bantam', u'targa': 313},
 u'cognome': u'Paperino',
 u'familiari': [{u'nome': u'Qui'}, {u'nome': u'Quo'}, {u'nome': u'Qua'}],
 u'genere': u'M',
 u'nome': u'Paolino',
 u'prima_apparizione': 1934}

## Alternativa: denormalizzazione

- Potrebbe non essere drammatico, ove dovesse essere necessario, denormalizzare i dati, duplicando parte dell'informazione
- in teoria bisogna tenere conto del limite di 64 MByte per documento, anche se nella pratica tale limite è spesso ben più che abbondante (e.g., un post di un blog implementato come un documento che incorpora i documenti che corrispondono ai commenti, tenendo conto che il testo dell'Amleto occupa 200KByte...)
- il prezzo da pagare consiste nel dover aumentare il numero di query (per esempio se si aggiornano dati duplicati)

## Come al solito...

...non c'è una soluzione universale, e molto spesso la scelta dipende oltre che dal problema che si vuole risolvere dal proprio stile di progettazione.

## C'è molto di più

- Viste
- Aggregation
- Dati geospaziali
- Replication
- Sharding (horizontal scaling)
- Ricerca testuale
- Sicurezza
